<a href="https://colab.research.google.com/github/snam1235/rnn_text_generation/blob/master/CuDNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import tensorflow as tf
import os
import tensorflow_datasets as tfds
from google.colab import files
import numpy as np
import pandas as pd
import sys 
import io
import re
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
#tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

Found GPU at: /device:GPU:0


In [92]:
# process the dataset:
chars = "abcdefghijklmnopqrstuvwxyz 0123456789"
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((c, i) for i, c in enumerate(chars))
n_vocab = len(chars)
print('Total Vocab : ', n_vocab) # number of unique characters

uploaded = files.upload()

lyric_file_names = ['cleaned_KanyeLowercase.txt', 'cleaned_taylorLyrics.txt']#, 'cleaned_PinkFloydLyrics','cleaned_HamiltonLyrics','cleaned_SelenaLyrics']

int_artist = dict((i, a) for i, a in enumerate(['Kanye West', 'Taylor Swift']))#, 'Pink Floyd', 'Hamilton', 'Selena Gomez']))

seq_len = 100
lyric_limit = 1000
data_X = []
data_y = []

n_chars = 0
for (i, lyric_file) in enumerate(lyric_file_names):
  with open(lyric_file, encoding='UTF_8') as fr:
    lyric_count = 0
    for lyric_chunk in fr:
      lyric_count = lyric_count + 1
      if lyric_count >= lyric_limit:
        break

      lyric_chunk = lyric_chunk.replace("\n", "").ljust(seq_len)
      if len(lyric_chunk) != seq_len:
        continue

      n_chars = n_chars + len(lyric_chunk)
      data_X.append([chars_int[char] for char in lyric_chunk])
      data_y.append(i)

print('Total Characters : ', n_chars) # number of all the characters in lyricsText.txt

# Store targets in data_y
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

Total Vocab :  37


Saving cleaned_KanyeLowercase.txt to cleaned_KanyeLowercase (17).txt
Saving cleaned_taylorLyrics.txt to cleaned_taylorLyrics (16).txt
Total Characters :  199800
Total Patterns :  1998


In [95]:
# batch_size = n_patterns
# hidden_size = 3

# encoder_inputs = Input(batch_shape=(batch_size, en_timesteps, en_vsize), name='encoder_inputs')
# decoder_inputs = Input(batch_shape=(batch_size, fr_timesteps - 1, fr_vsize), name='decoder_inputs')

# encoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, name='encoder_gru')
# encoder_out, encoder_state = encoder_gru(encoder_inputs)

# decoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, name='decoder_gru')
# decoder_out, decoder_state = decoder_gru(decoder_inputs, initial_state=encoder_state)

# attn_layer = AttentionLayer(name='attention_layer')
# attn_out, attn_states = attn_layer([encoder_out, decoder_out])

# decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_out, attn_out])

# dense = Dense(fr_vsize, activation='softmax', name='softmax_layer')
# dense_time = TimeDistributed(dense, name='time_distributed_layer')
# decoder_pred = dense_time(decoder_concat_input)

# full_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred)
# full_model.compile(optimizer='adam', loss='categorical_crossentropy')

LSTM_layer_num = 4 # number of LSTM layers
layer_size = [256] * LSTM_layer_num # number of nodes in each layer

model = Sequential()
model.add(LSTM(layer_size[0], input_shape=(X.shape[1], X.shape[2]), return_sequences = True))
for i in range(1,LSTM_layer_num):
    model.add(LSTM(layer_size[i], return_sequences=True))

model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()
# Configure the checkpoint :
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_43 (LSTM)               (None, 100, 256)          264192    
_________________________________________________________________
lstm_44 (LSTM)               (None, 100, 256)          525312    
_________________________________________________________________
lstm_45 (LSTM)               (None, 100, 256)          525312    
_________________________________________________________________
lstm_46 (LSTM)               (None, 100, 256)          525312    
_________________________________________________________________
flatten_13 (Flatten)         (None, 25600)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 51202     
_________________________________________________________________
activation_13 (Activation)   (None, 2)               

In [96]:
# Fit the model :
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}
model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])
# Load weights file :
weights_file = './models/Weights-LSTM-improvement-004-2.49538-bigger.hdf5' # weights file path
model.load_weights(weights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

Train on 1598 samples, validate on 400 samples
Epoch 1/30
1598/1598 [==============================] - 15s 9ms/step - loss: 0.6883 - val_loss: 0.8492

Epoch 00001: loss improved from inf to 0.68828, saving model to Weights-LSTM-improvement-001-0.68828-bigger.hdf5
Epoch 2/30
1598/1598 [==============================] - 13s 8ms/step - loss: 0.6662 - val_loss: 0.8908

Epoch 00002: loss improved from 0.68828 to 0.66619, saving model to Weights-LSTM-improvement-002-0.66619-bigger.hdf5
Epoch 3/30
1598/1598 [==============================] - 13s 8ms/step - loss: 0.6633 - val_loss: 1.0072

Epoch 00003: loss improved from 0.66619 to 0.66331, saving model to Weights-LSTM-improvement-003-0.66331-bigger.hdf5
Epoch 4/30
1598/1598 [==============================] - 13s 8ms/step - loss: 0.6640 - val_loss: 0.8759

Epoch 00004: loss did not improve from 0.66331
Epoch 5/30
1598/1598 [==============================] - 13s 8ms/step - loss: 0.6629 - val_loss: 1.0179

Epoch 00005: loss improved from 0.66331

KeyboardInterrupt: ignored

In [101]:
# Set a random seed:
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
pattern_text = ''.join([int_chars[value] for value in pattern])
print('Testing: ')
print("\"",''.join(pattern_text), "\"\n")

x = np.reshape(pattern, ( 1, len(pattern), 1))
x = x / float(n_vocab)
prediction = model.predict(x,verbose = 0)
for i in range(len(prediction[0])):
  print(int_artist[i] + ": " + str(prediction[0][i]))
  
index = np.argmax(prediction)
result = int_artist[index]
sys.stdout.write("Predicted " + result + " wrote: " + pattern_text)
print('\nDone')

Testing: 
" gone long grab that ass shed your clothes say you long for me for you lay it off with all your rules "

Kanye West: 0.6397787
Taylor Swift: 0.36022136
Predicted Kanye West wrote: gone long grab that ass shed your clothes say you long for me for you lay it off with all your rules
Done


In [0]:

# import re, string;
# pattern = re.compile('[\W_]+')

# dictionary_file_name = 'intersected_dictionary.txt'
# with open(dictionary_file_name, encoding='UTF_8') as fr:
#   dictionary_text = [line for line in fr][:405]
  
# textFileName = 'KanyeLowercase.txt'
# raw_text = open(textFileName, encoding = 'UTF-8').read()
# raw_text = raw_text.replace("\n", " ").lower().split(" ")
# raw_text = [pattern.sub('', raw_word) for raw_word in raw_text]
# raw_text.remove('')